In [45]:
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm catboost imbalanced-learn

In [46]:
import joblib
import pandas as pd
import numpy as np

# Load required libraries for predictions
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression


In [49]:
import joblib
import pandas as pd
import numpy as np

# === Load the saved scaler ===
scaler_pe = joblib.load('/kaggle/input/scalers/scaler_pe.pkl')  

# === Load the saved ensemble models ===
ensemble_models_pe = joblib.load('/kaggle/input/dependencies/ensemble_model_pe.pkl')

# === Extract individual models from the ensemble ===
best_xgb_pe = ensemble_models_pe['best_xgb']
best_lgbm_pe = ensemble_models_pe['best_lgbm']
meta_model_pe = ensemble_models_pe['meta_model']

print("Models and scaler loaded successfully!")

# === Load the PE test data ===
test_data_pe = pd.read_csv("/kaggle/input/crest-dataset/dataset/test.csv")  # Replace with the actual test file path

# === Load selected features ===
selected_features_pe = pd.read_csv("/kaggle/input/dependencies/selected_features_pe.csv", header=None)
selected_features_pe = selected_features_pe.iloc[1:][0].tolist()  # Remove header row if necessary

# === Select only relevant columns based on training data features ===
test_selected_features_pe = test_data_pe[selected_features_pe]

# === Apply log transformation & handle NaN values ===
test_selected_features_pe = np.log1p(test_selected_features_pe)  
test_selected_features_pe = np.nan_to_num(test_selected_features_pe)

# === Apply the loaded scaler ===
test_scaled_pe = scaler_pe.transform(test_selected_features_pe)

print("Test data preprocessed successfully!")

# === Get probabilities from base models (XGBoost and LightGBM) ===
xgb_probs_pe = best_xgb_pe.predict_proba(test_scaled_pe)  # Get probabilities from XGBoost
lgbm_probs_pe = best_lgbm_pe.predict_proba(test_scaled_pe)  # Get probabilities from LightGBM

# === Create metadata for Meta-Model ===
metadata_df_pe = pd.DataFrame(xgb_probs_pe, columns=[f"xgb_class_{i}" for i in range(xgb_probs_pe.shape[1])])
metadata_df_pe = metadata_df_pe.join(pd.DataFrame(lgbm_probs_pe, columns=[f"lgbm_class_{i}" for i in range(lgbm_probs_pe.shape[1])]))

# === Check if the meta-model is expecting the correct number of features ===
print("Metadata shape:", metadata_df_pe.shape)  # This should match the expected number of features in the meta-model

# === Predict final probabilities using Meta-Model ===
final_probs_pe = meta_model_pe.predict_proba(metadata_df_pe)  # Get final probabilities from meta-model

# === Convert probabilities to final class predictions ===
final_predictions_pe = np.argmax(final_probs_pe, axis=1)  # Convert to final class predictions by taking the max probability

# === Prepare and save the final predictions ===
final_predictions_df = pd.DataFrame({
    "SHA256": test_data_pe["SHA256"],  # Include SHA256 for traceability
    "Final_Predicted_Class": final_predictions_pe
})

# Save the final predictions
final_predictions_df.to_csv("final_predictions_pe.csv", index=False)
print("Predictions generated and saved to 'final_predictions_pe.csv'!")

# === Save the metadata for future use ===
metadata_df_pe.to_csv("metadata_pe.csv", index=False)
print("Metadata saved to 'metadata_pe.csv' for meta-model training!")


Models and scaler loaded successfully!
Test data preprocessed successfully!
Metadata shape: (1480, 14)
Predictions generated and saved to 'final_predictions_pe.csv'!
Metadata saved to 'metadata_pe.csv' for meta-model training!


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  class attribute, which is a dictionary `param_name: list of constraints`. See


In [57]:
import pandas as pd
import numpy as np

# === Load the test metadata (PE in this case) ===
metadata_pe_test = pd.read_csv("/kaggle/working/metadata_pe.csv")  # PE metadata for test

# === Load the saved meta-model (already trained) ===
meta_model = meta_model_pe

# === Get predictions using the meta-model ===
final_predictions_test = meta_model.predict(metadata_pe_test)

# === Get class probabilities (in case you want to do argmax on them) ===
# If you're using probabilities for final predictions (if required)
# final_predictions_prob = meta_model.predict_proba(metadata_pe_test)

# === Prepare the final output (SHA256 and label) ===
test_data = pd.read_csv("/kaggle/input/crest-dataset/dataset/test.csv")  # Assuming original test data is available for SHA256
test_submit = pd.DataFrame({
    "SHA256": test_data["SHA256"],  # Use the SHA256 column from the original test data
    "label": final_predictions_test  # Use the predicted labels from the meta-model
})

# === Save the predictions to test_submit.csv ===
test_submit.to_csv("/kaggle/working/test_submit.csv", index=False)
print("Submission file 'test_submit.csv' created successfully!")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  class attribute, which is a dictionary `param_name: list of constraints`. See


Submission file 'test_submit.csv' created successfully!


ImportError: cannot import name '_joblib_parallel_args' from 'sklearn.utils.fixes' (/usr/local/lib/python3.10/dist-packages/sklearn/utils/fixes.py)